In [4]:
! pip install keras
import numpy as np
from keras.preprocessing import image
from keras import backend as K
from PIL import Image
import tensorflow as tf
import tensorflow.keras.backend as K
import torch

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from skimage.io import imread
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from utils import blackbox_inference, submit

token = "8ZYRkMBfW1"

In [8]:
model = load_model('model-final.h5')

In [101]:
im = imread("Images/whitebox/1.ppm")
im_pil = Image.fromarray(im)
im_tf = tf.convert_to_tensor(np.array([im]), dtype=tf.float32)

im_tf

<tf.Tensor: shape=(1, 139, 139, 3), dtype=float32, numpy=
array([[[[128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.],
         ...,
         [128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.]],

        [[128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.],
         ...,
         [128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.]],

        [[128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.],
         ...,
         [128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.]],

        ...,

        [[128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.],
         ...,
         [128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.]],

        [[128., 128., 128.],
         [128., 128., 128.],
         [128., 128., 128.],
         ...,
         [128., 128., 128.],
         [128., 128., 128.],
         

In [6]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [102]:
proba = model.predict(im_tf)

In [103]:
probabilities = model.predict(np.array([im]))[0]
model_input_layer = model.layers[0].input
model_output_layer = model.layers[-1].output
probabilities = model.predict(model_input_layer)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [120]:
new_proba = np.delete(probabilities, [np.argmax(probabilities)])
target_class = np.argmax(new_proba)
loss = -model.predict(im_tf)[0][target_class]
target_class, loss

(4, -0.18459061)

In [121]:
gradients = K.gradients(model.output, model.input)

# Wrap the input tensor and the gradient tensor in a callable function
f = K.function([model.input], gradients)

# Random input image
x = np.random.rand(1, 100,100,3)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [116]:
with tf.GradientTape() as t:
    sess = tf.InteractiveSession()
    loss = -model.predict(im_tf)[0][target_class]
    t.gradient(loss, im_tf)
    sess.run(tf.initialize_all_variables())
    evaluated_gradients = sess.run(gradients,feed_dict={model.input:im_tf})

    


AttributeError: module 'tensorflow' has no attribute 'InteractiveSession'

In [119]:
with tf.GradientTape() as t:
    outputTensor = model.predict(im_tf)
    listOfVariableTensors = im_tf
    bce = -outputTensor[0][target_class]
    gradients = t.gradients(loss, listOfVariableTensors)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    evaluated_gradients = sess.run(gradients,feed_dict={model.input:im_tf})
    print(evaluated_gradients)

LookupError: No gradient defined for operation 'IteratorGetNext' (op type: IteratorGetNext)

In [29]:
np.shape(im)


(139, 139, 3)